<a href="https://colab.research.google.com/github/pea-sys/Til/blob/master/CategoricalVariablesEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview
カテゴリ変数の取り扱いについての勉強ノート


## Motivation
Kaggleに取り組むにあたり学んだあれこれをアウトプットして、知識を整理する。

[参考URL](https://github.com/h2oai/h2o-meetups/blob/master/2017_11_29_Feature_Engineering/Feature%20Engineering.pdf)

まずは検証データを作成

In [0]:
import pandas as pd

In [50]:
df = pd.DataFrame(["A","A","A","A","B","B","B","C","C"],columns=['Feature'],dtype='category')
df

,Feature
0,A
1,A
2,A
3,A
4,B
5,B
6,B
7,C
8,C


### Label Encoding
単純に各カテゴリ値にユニークな正数を割り当てる。
エンコード結果は１列となるため、メモリ効率は良い。    
大小関係のない名義尺度の変換に使用するのはＮＧ。
決定木ベースのモデル以外では使用しない方が良いとのこと。  

In [51]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le = le.fit(df['Feature'])
df['LabelEncoder'] = le.transform(df['Feature'])
df

,Feature,LabelEncoder
0,A,0
1,A,0
2,A,0
3,A,0
4,B,1
5,B,1
6,B,1
7,C,2
8,C,2


### One Hot Encoder
カテゴリ数分の要素を確保するベクトル表現。  
該当する要素が1となり、それ以外は0になる(One Hot)。  
カテゴリ変数間に数的関係が見いだせないようなデータに使用する（ほとんどがそう）。
決定木ベースモデル以外で使用する。  
回帰分析等で使用する場合は、多重共線性を避けるため、１つカラムをドロップするらしい  (全て0=ドロップしたカラム1)  
デメリットはカテゴリ数 * 行だけデータが増加するので、カテゴリ数が多いとメモリ消費量がネックとなる。  
メモリは開発環境にもよるがkaggleのkernelだと700個程度のカラムデータを扱ってメモリエラーになったりするので注意が必要。  
個人的に気になるのが、OneHotEncoderが適用されたカラムが複数ある場合に、
元のカテゴリグループをモデルが見分けることが出来ないんじゃないか？という心配もある。  
見分ける必要がない場合も多いとは思うけど、学習データが大量ではない場合に怖い気もする。  
時々、OHEと略記表記されているのを見かける。  
pandasのget_dummiesもOHEみたいな動きだが、これはデータフレームに対して行うため、  
訓練データまたは評価データにしか現れないカテゴリ値がある場合に一旦結合しないといけないため、 
はっきりいってダサイし、リスクがある。  




In [52]:
! pip install category_encoders

In [53]:
#sikit-learnのOHEは一旦カテゴリ値を数値にしないといけないので面倒
#社会的には新しいライブラリをインストールする方がリスキーかもしれないが
import category_encoders as ce
list_cols = ['Feature']
ce_ohe = ce.OneHotEncoder(cols=list_cols)
df = pd.concat([df,ce_ohe.fit_transform(df['Feature'])],axis=1)
df


,Feature,LabelEncoder,Feature_1,Feature_2,Feature_3
0,A,0,1,0,0
1,A,0,1,0,0
2,A,0,1,0,0
3,A,0,1,0,0
4,B,1,0,1,0
5,B,1,0,1,0
6,B,1,0,1,0
7,C,2,0,0,1
8,C,2,0,0,1


### Count Encoding
データに含まれるカテゴリ変数の出現回数を数える。  
系列的な特徴を作りたい時に使うかも  
予兆や再発予知、リピータ検出などに良いかもしれない。  
モデルに学習させるための積極的なエンコーディングだと思う。  
テストデータのLeakがあるので、ホールドアウトしてから行う。
KaggleのKernelだとバリデーション切る前にやってるのが結構ある。  
多分ダメだと思う。  
が、私もまだ機械学習を学んで半年なので自信はない。

In [54]:
df['Feature_Count'] = df.groupby('Feature')['LabelEncoder'].transform('count')
df

,Feature,LabelEncoder,Feature_1,Feature_2,Feature_3,Feature_Count
0,A,0,1,0,0,4
1,A,0,1,0,0,4
2,A,0,1,0,0,4
3,A,0,1,0,0,4
4,B,1,0,1,0,3
5,B,1,0,1,0,3
6,B,1,0,1,0,3
7,C,2,0,0,1,2
8,C,2,0,0,1,2


### LabelCount Encoding
カテゴリの出現回数を調べ、順位を特徴量とする。  
どういう時に使うか直感的理解は得ていない。  
頻出データの重み係数を小さくする時に役立ったりするのかもしれない？


In [55]:
count_rank = df.groupby('Feature')['LabelEncoder'].count().rank(ascending=False)
df['Feature_LabelCount'] = df['Feature'].map(count_rank)
df

,Feature,LabelEncoder,Feature_1,Feature_2,Feature_3,Feature_Count,Feature_LabelCount
0,A,0,1,0,0,4,1.0
1,A,0,1,0,0,4,1.0
2,A,0,1,0,0,4,1.0
3,A,0,1,0,0,4,1.0
4,B,1,0,1,0,3,2.0
5,B,1,0,1,0,3,2.0
6,B,1,0,1,0,3,2.0
7,C,2,0,0,1,2,3.0
8,C,2,0,0,1,2,3.0


# 工事中
まだまだエンコーディング手法はあります

## 感想
エンコーディングはモデルが学習しやすいように行うことが肝要。
モデルによってエンコーディング方法を変えるといったことは、  
書籍では見受けられなかったが  Kaggleでは当たり前に行われている。  
直感的理解とは裏腹に意外な特徴が効いたりするので、結局やれることはなるべく全部やりましょうってことになりそう。

#### 疑問点
* オンライン学習などでカテゴリ変数が増える可能性がある場合、OHEは使えないのではないか(学習しなおし）？
* カテゴリ変数をサポートしているモデルは内部で、どのエンコーディングを使用しているのか
